# Lab 4. Introducere PyTorch

### Ce este PyTorch?

PyTorch este o biblioteca Python folosita cu doua scopuri:
 * reprezinta un inlocuitor al bibliotecii NumPy, care se poate folosi de puterea computationala a componentelor precum GPUs
 * contine implementarea unui sistem automat de derivare, util pentru antrenarea retelelor neuronale

In [ ]:
# ! pip install torch torchvision

## Tensori

Tensorii sunt o structura de date speciala, asemanatoare vectorilor si matricelor. Tensorii sunt folositi in PyTorch pentru a stoca inputul si outputul modelelor, dar si parametrii modelelor.

Tensorii se aseamana foarte mult cu NumPy arrays, cu diferenta ca pot fi procesati de GPU sau alte tipuri de hardware specializat.

In [1]:
import torch
import numpy as np

### Initializarea tensorilor

**Folosind Python lists**

In [2]:
data = [[1, 2], [3, 4]]
t_from_list = torch.tensor(data)
print(t_from_list)

tensor([[1, 2],
        [3, 4]])


**Folosind NumPy arrays**

In [3]:
np_array = np.array(data)
t_from_np = torch.from_numpy(np_array)
print(t_from_np)

tensor([[1, 2],
        [3, 4]])


**Folosind un alt tensor**

Noul tensor retine proprietatile (shape + tip de date) ale tensorului original, daca nu este specificat explicit altfel.

In [4]:
t_ones = torch.ones_like(t_from_list)  # aceleasi proprietati cu ale lui t_from_list
print(f"Tensor plin de 1:\n{t_ones}\n")

t_rand = torch.rand_like(t_from_list, dtype=torch.float)  # suprascrie tipul de date
print(f"Tensor random:\n{t_rand}\n")

Tensor plin de 1:
tensor([[1, 1],
        [1, 1]])

Tensor random:
tensor([[0.2685, 0.9653],
        [0.8737, 0.9983]])



**Folosind valori aleatoare sau constante**

In [5]:
shape = (2, 3)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random tensor:\n{rand_tensor}\n")
print(f"Ones tensor:\n{ones_tensor}\n")
print(f"Zeros tensor:\n{zeros_tensor}\n")

Random tensor:
tensor([[0.1772, 0.1739, 0.9019],
        [0.8926, 0.6885, 0.9407]])

Ones tensor:
tensor([[1., 1., 1.],
        [1., 1., 1.]])

Zeros tensor:
tensor([[0., 0., 0.],
        [0., 0., 0.]])



### Proprietatile tensorilor

In [6]:
tensor = torch.rand(3, 4)

print(f"Shape: {tensor.shape}")
print(f"Tip de date: {tensor.dtype}")
print(f"Device: {tensor.device}")

Shape: torch.Size([3, 4])
Tip de date: torch.float32
Device: cpu


### Operatii cu tensori

Puteti gasi [aici](https://pytorch.org/docs/stable/torch.html) o lista exhaustiva cu operatii (transpuneri, indexari, slicing, concatenari, operatii matematice, etc.)

Toate aceste operatii pot fi realizate mai rapid cu ajutorul unui GPU.

In [7]:
# mutam tensorul pe GPU daca avem unul la dispozitie
if torch.cuda.is_available():
    tensor = tensor.to("cuda")
    print(f"Tensorul este stocat acum pe: {tensor.device}")

Tensorul este stocat acum pe: cuda:0


**Indexdare si slicing (asemanator NumPy)**

In [8]:
tensor = torch.ones(4, 4)
tensor[:, 1] = 0
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


**Concatenare**

In [9]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


**Inmultirea tensorilor**

In [10]:
# inmultire "element cu element"
tensor = torch.tensor([[0, 2], [2, 0], [2, 2]])
print(f"tensor.mul(tensor)\n{tensor.mul(tensor)}\n")
print(f"tensor * tensor\n{tensor * tensor}\n")

tensor.mul(tensor)
tensor([[0, 4],
        [4, 0],
        [4, 4]])

tensor * tensor
tensor([[0, 4],
        [4, 0],
        [4, 4]])



In [11]:
# inmultire de matrici
# inmultim matricea cu transpusa (tensor.T)
print(f"tensor.matmul(tensor.T)\n{tensor.matmul(tensor.T)}\n")
print(f"tensor @ tensor.T\n{tensor @ tensor.T}\n")

tensor.matmul(tensor.T)
tensor([[4, 0, 4],
        [0, 4, 4],
        [4, 4, 8]])

tensor @ tensor.T
tensor([[4, 0, 4],
        [0, 4, 4],
        [4, 4, 8]])



**Operatii in-place**

Operatiile care au `_` ca sufix modifica tensorul pentru care au fost apelate. Exemplu: `x.copy_(y)` sau `x.t_()` il modifica pe `x`.

Aceste metode salveaza memorie insa aduc probleme atunci cand vine vorba de calculul automat al derivatelor operatiilor efectuate (de aceea folosirea lor NU este recomandata).

In [12]:
print(tensor, "\n")
tensor.add_(5)
print(tensor)

tensor([[0, 2],
        [2, 0],
        [2, 2]]) 

tensor([[5, 7],
        [7, 5],
        [7, 7]])


### Legatura cu NumPy

Un tensor stocat pe CPU si un NumPy array pot imparti aceeasi memorie in program, astfel modificarile asupra unuia se reflecta asupra celuilalt.

In [13]:
# tensor -> np array
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]
t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


In [14]:
# np array -> tensor
n = np.ones(5)
t = torch.from_numpy(n)
print(f"t: {t}")
print(f"n: {n}")

np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
n: [1. 1. 1. 1. 1.]
t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]


## torch.autograd

`torch.autograd` este motorul de derivare automata folosit de PyTorch pentru antrenarea retelelor neuronale.

### Retele neuronale

Retelele neuronale (NN - Neural Networks) sunt o colectie de functii inlantuite care sunt aplicate asupra unor date de intrare. Aceste functii se definesc prin parametrii (weights and biases), care in PyTorch sunt stocate prin intermediul tensorilor.

Antrenarea unei retele neuronale se face in doi pasi:
 - **forward propagation**: reteaua incearca sa prezica output-ul corect pe baza datelor de intrare. Paseaza datele de intrare prin sirul de functii (input-ul functiei urmatoare este output-ul functiei curente din lista)
 - **backward propagation**: in acest pas parametrii retelei sunt ajustati proportional cu eroare produsa de predictia facuta la pasul forward. Acest lucru este realizat prin a parcurge lista de functii invers, pornind de la output, calculand la fiecare pas derivata erorii in raport cu parametrii functiilor (acestor derivate le mai spunem gradienti), iar parametrii sunt optimizati folosind coborarea pe gradient (gradient descent).
 
### Implementarea in PyTorch

Vom incerca un singur pas de antrenare. Incarcam un model preantrenat `resnet18` folosit pentru clasificarea de imagini in $1000$ de clase. Cream un tensor random pentru o singura imagine de $64 \times 64$ pixeli cu $3$ canale de culoare. Initializam label-ul acestei imagini cu un tensor random de dimensiune $(1, 1000)$.

In [15]:
import torchvision

# vom considera pentru moment acest model ca un black-box
# care primeste ca input tensori de forma (batch_size, 3, 64, 64)
# si returneaza pentru fiecare imagine din batch distributia labelurilor pentru acea imagine
# forma outputului este deci: (batch_size, 1000)
model = torchvision.models.resnet18(pretrained=True)

data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

In [16]:
# pasul forward
prediction = model(data)

Folosim predictia modelului si labelul corect pentru a calcula eroarea (`loss`). Pasul urmator este propagarea inapoi (backpropagation) a acestei erori prin retea. Acest lucru se realizeaza cand apelam metoda `.backword()` pentru tensorul ce contine eroarea. Motorul `autograd` calculeaza si stocheaza gradientii pentru fircare parametru al modelului in atributul `.grad` al acestor parametrii.

In [17]:
loss = (prediction - labels).sum()
loss.backward()  # pasul backward

Apoi, incarcam un algoritm de optimizare a parametrilor (optimizer), in cazul de fata [SGD](https://towardsdatascience.com/stochastic-gradient-descent-with-momentum-a84097641a5d) (Stochastic Gradient Descent) cu learning rate de $0.01$ si momentum $0.9$.

Mai intai inregistram toti parametrii modelului in optimizer.

In [18]:
optim = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In cele din urma, apelam `.step()` pentru optimizer pentru a initia algoritmul de coborare pe gradient. Optimizerul ajusteaza astfel fiecare parametru al modelului pe baza gradientilor calculati la pasul backward.

In [19]:
optim.step()  # gradient descent

[Alti](https://pytorch.org/docs/stable/optim.html) optimizeri utili.

### Derivarea automata in Autograd

In exemplele urmatoare vom vedea cum `autograd` colecteaza si calculeaza gradientii. Cream doi tensori cu atributul `requires_grad` setat pe `True`. Acest lucru semnalizeaza motorului `autograd` ca toate operatiile in care sunt implicati acesti tensori trebuie urmarite (tracked).

In [20]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Cream acum un tensor care retine o operatie intre `a` si `b`.
$$Q = 3a^3-b^2$$

In [21]:
Q = 3 * a**3 - b**2

Daca presupunem ca $a$ si $b$ ar fi parametrii unei retele si $Q$ ar fie eroarea, la antrenare ne intereseaza derivata erorii in raport cu fiecare parametru in parte:

$$\frac{\partial Q}{\partial a} = 9a^2$$

$$\frac{\partial Q}{\partial b} = -2b$$

Cand apelam `.backward()` pentru `Q`, `autograd` calculeaza acesti gradienti si ii salveza in atributul `.grad` al tensorilor corespunzatori.

In acest exemplu trebuie sa pasam explicit un argument `gradient` functiei `Q.backward()` intrucat `Q` este vector. `gradient` este un tensor de aceeasi forma ca si `Q` si reprezinta derivata lui $Q$ in raport cu el insusi:

$$\frac{\partial Q}{\partial Q} = 1$$

Alternativ, putem agrega $Q$ intr-o valoare scalara si sa aplicam `backward` fara a mai specifica gradientul initial: `Q.sum().backward()`.

In [22]:
# Q este vector specificam derivata cu el insusi
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

# agregam Q intr-un scalar
# Q.sum().backward()

Gradientii sunt stocati acum in `a.grad` si `b.grad`.

In [23]:
print(f"9a^2 = {9 * a ** 2}")
print(f"dQ/da = {a.grad}\n")
print(f"-2b = {-2 * b}")
print(f"dQ/db = {b.grad}")

9a^2 = tensor([36., 81.], grad_fn=<MulBackward0>)
dQ/da = tensor([36., 81.])

-2b = tensor([-12.,  -8.], grad_fn=<MulBackward0>)
dQ/db = tensor([-12.,  -8.])


### Graful computatiilor

Conceptual, `autograd` retine tensorii si toate operatiile executate asupra acestora, impreuna cu rezultatele intermediare, sub forma unui graf orientat aciclic (DAG - Directed Acyclic Graph) in care nodurile sunt obiecte de tip [Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function). In acest DAG, frunzele sunt tensorii din input, iar radacina/radacinile sunt tensorii output. Parcurgand drumuri in acest graf de la radacini la frunze, gradientii pot fi calculati automat folosind "chain rule":

$$\frac{dz}{dx} = \frac{dz}{dy} \cdot \frac{dy}{dx}$$

In pasul forward, `autograd`:
 * calculeaza rezultatul operatiei curente intr-un tensor
 * mentine functia derivata a acestei operatii in DAG, in atributul `.grad_fn` al tensorului rezultat
 
Pasul backward incepe cand este apelata metoda `.backward()` pentru o radacina din DAG. Atunci, `autograd`:
 * calculeaza fiecare gradient folosind `.grad_fn`
 * acumuleaza gradientul in atributul `.grad` al tensorului respectiv
 * folosind "chain rule" propaga gradientii pana la frunze
 
Reprezentarea DAG-ului pentru calculul de mai devreme al lui `Q` este aceasta:

![](https://pytorch.org/tutorials/_images/dag_autograd.png)

In [24]:
fn = Q.grad_fn
print(fn)
print(fn.next_functions)

((<MulBackward0 object at 0x7f50f873ddc0>, 0), (<PowBackward0 object at 0x7f50f873d5b0>, 0))


### Excluderea din DAG

`autograd` urmareste operatiile efectuate pe toti tensorii care au `requires_grad=True`. Pentru tensorii ca nu au nevoie de gradienti, setarea acestui atribut cu `False` ii exclude din DAG.

Tensorul output al unei operatii va avea `requires_grad=True` daca macar un tensor input are de asemenea `requires_grad=True`.

In [25]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"a.requires_grad: {a.requires_grad}")
b = x + z
print(f"b.requires_grad: {b.requires_grad}")

a.requires_grad: False
b.requires_grad: True


## Pasii de antrenare a unui model

Vom ilustra pasii ce trebuie urmati in cazul in care vrem sa definim si antrenam un model pentru o problema simpla de clasificare.

Mai intai vom defini datele noastre, un set foarte mic de date clasificate in 3 clase.

In [26]:
dummy_data = {
    "vectors": [
        torch.tensor([0., 0., 0., 0., 0., 0., 2., 3., 4.]),
        torch.tensor([0., 0., 0., 0., 0., 0., 4., 2., 5.]),
        torch.tensor([0., 0., 0., 0., 0., 0., 7., 9., 1.]),
        torch.tensor([0., 0., 0., 4., 5., 2., 0., 0., 0.]),
        torch.tensor([0., 0., 0., 2., 4., 6., 0., 0., 0.]),
        torch.tensor([0., 0., 0., 2., 3., 5., 0., 0., 0.]),
        torch.tensor([2., 9., 6., 0., 0., 0., 0., 0., 0.]),
        torch.tensor([6., 5., 4., 0., 0., 0., 0., 0., 0.]),
        torch.tensor([7., 7., 4., 0., 0., 0., 0., 0., 0.]),
    ],
    "labels": [0, 0, 0, 1, 1, 1, 2, 2, 2],
}

### PyTorch Dataset

Vom incarca setul nostru de date intr-un obiect din clasa [Dataset](https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset).

In [27]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data
        
    # urmatoarele doua metode trebuie implementate
    def __getitem__(self, k):
        """Returneaza al k-lea exemplu din dataset"""
        return (
            self.data["vectors"][k],
            self.data["labels"][k]
        )
    
    def __len__(self):
        """Returneaza dimensiunea datasetului"""
        return len(self.data["labels"])

In [28]:
dummy_dataset = MyDataset(dummy_data)
len(dummy_dataset)

9

### PyTorch DataLoader

In timpul antrenarii vrem sa parcurgem exemplele din dataset si sa le trecem prin reteaua noastra. In practica, mai multe exemple sunt trecute prin retea inainte de a apela pasul backward. Un astfel de grup se numeste batch.

DataLoaderul este un iterator construit plecand de la dataset, care grupeaza mai multe exemple in batch-uri.

In [29]:
dummy_dataloader = torch.utils.data.DataLoader(dummy_dataset, batch_size=2, shuffle=True)

In [30]:
next(iter(dummy_dataloader))

[tensor([[0., 0., 0., 0., 0., 0., 7., 9., 1.],
         [6., 5., 4., 0., 0., 0., 0., 0., 0.]]),
 tensor([0, 2])]

### Definitia modelului

Dupa cum am spus mai devreme, un model este o inlantuire de functii (sau straturi/layere). Pentru a defini un model implementam o clasa care mosteneste clasa `torch.nn.Module` si implementam constructorul acesteia in care definim layerele si metoda `forward` in care descriem cum sunt aplicate aceste layere.

Vom implementa un model foarte simplu: un singur layer linear (feed-forward) care primeste un vector de dimensiune $9$ (numarul de features ale datelor noastre) si returneaza un output de dimensiune $3$ (numarul de clase din problema noastra).

Un layer linear este o operatie de forma $y = A \cdot x + b$ (unde matricea $A$ reprezinta weight-urile layerului, iar vectorul $b$, biasul).

In [31]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        output = self.linear(x)  # aplicam layerul linear
        return output

### Calculul erorii

Pentru un vector de dimensiune $9$ modelul returneaza unul de dimensiune $3$. Cum putem interpreta acest vector pentru a obtine predictiile?

Am putea spre exemplu sa determinam pozitia valorii maxime si sa spunem ca aceea este clasa prezisa de model

In [32]:
t = torch.tensor([3., 4., 1.])
print("Maxim:\t\t", t.max(0)[0])
print("Poz. maxim:\t", t.max(0)[1])

Maxim:		 tensor(4.)
Poz. maxim:	 tensor(1)


Insa o varianta mai buna ar fi sa transformam valorile tensorului intr-o distributie de probabilitate (cu ce probabilitate atribuie modelul clasa $i$ exemplului?).

O functie care calculeaza o astfel de distributie pornind de la un tensor oarecare este functia [*softmax*](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html).

In [33]:
print(torch.nn.functional.softmax(t, dim=0))

tensor([0.2595, 0.7054, 0.0351])


Folosind distributia generata de model si labelurile adevarate putem calcula acum eroarea. O functie de eroare des folosita pentru clasificare este [*cross-entropy*](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

In [34]:
loss_fn = torch.nn.CrossEntropyLoss()

# presupunem ca aceasta este output-ul modelului pentru un batch de 2 exemple
output = torch.tensor([
    [3., 7., 4.],
    [4., 2., 3.],
])

# si presupunem ca aceastea ar fi labelurile adevarate
true_labels = torch.tensor([1, 1])

print("Labeluri prezise:", output.max(1)[1])

# functia de loss aplica automat softmax peste output-ul modelului
# rezultatul este un scalar corespunzator erorii medii pentru toate exemplele din batch
print("loss: ", loss_fn(output, true_labels))

Labeluri prezise: tensor([1, 0])
loss:  tensor(1.2367)


### O epoca de antrenare

O epoca de antrenare reprezinta parcurgerea datasetului (o data) si actualizarea parametrilor modelului pe baza predictiilor facute. 

In [35]:
# cuda daca avem la dispozitie GPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# initializam modelul si il mutam pe device-ul corespunzator
model = Model().to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# setam modelul in modul de antrenare
model.train()

# parcurgem batch-urile folosind dataloaderul
for (vectors, labels) in dummy_dataloader:
    # mutam vectorii si labelurile pe device-ul corespunzator
    vectors = vectors.to(DEVICE)
    labels = labels.to(DEVICE)
    
    # reseteaza gradientii ramasi de la batch-ul anterior
    optimizer.zero_grad()
    
    # trecem vectorii (batch_size x 9) din batch prin model (forward step)
    output = model(vectors)  # shape: batch_size x 3
    
    # calculam loss-ul
    loss = loss_fn(output, labels)
    
    # backward step
    loss.backward()
    
    # actualizam parametrii modelului folosind optimizerul
    optimizer.step()

Acum putem evalua modelul

In [36]:
# trecem modelul in modul de evaluare
model.eval()

true_labels = []
predicted_labels = []
mean_loss = 0

with torch.no_grad():  # dezactivam engine-ul autograd
    for (vectors, labels) in dummy_dataloader:
        vectors = vectors.to(DEVICE)
        labels = labels.to(DEVICE)

        output = model(vectors)
        loss = loss_fn(output, labels)

        mean_loss += loss.item()
        true_labels.extend(labels.tolist())
        predicted_labels.extend(output.max(1)[1].tolist())

mean_loss /= len(dummy_dataloader)

In [37]:
import sklearn.metrics as metrics

print("Accuracy:", metrics.accuracy_score(true_labels, predicted_labels))
print("Mean loss:", mean_loss)

Accuracy: 1.0
Mean loss: 0.11842860709875822


#### Important !!!

Retelele neuronale pot obtine destul de usor performanta foarte buna pe datele cu care au fost antrenate, dar fara sa generalizeze (pe date noi, obtin performanta foarte slaba). Numim acest caz **overfit**.

Pentru a stabili clar performanta modelului in practica se realizeaza o impartire a datelor in 3 categorii:
 - train: datele pe care este antrenat modelul la fiecare epoca
 - validare: la finalul fiecarei epoci evaluam performanta modelului pe aceste date (nevazute in timpul antrenarii)
 - test: odata ce am antrenat cel mai bun model (selectat conform datelor de validare) evaluarea finala este facuta pe aceste date (de asemenea, tot nevazute in timpul antrenarii)
 
Datele de validare sunt folosite pentru a monitoriza performanta (loss, acuratete, etc.) modelului la finalul fiecarei epoci. Putem selecta de exemplu parametrii modelului de la finalul epocii cu cea mai bune performanta. De asemenea, daca observam ca eroare pe datele de validare stagneaza sau creste in timp ce eroarea pe datele de train continua sa scade, putem trage concluzia ca modelul a inceput sa faca **overfit** pe datele de train.

### Inlantuirea mai multor layere

Putem defini de exemplu mai multe layere feed-forward (lineare) si sa trecem input-ul succesiv prin ele.

In [38]:
t_input = torch.tensor([[1., 2., 3., 4., 5.]])  # intotdeauna avem shape (batch_size, dim)

layer_1 = torch.nn.Linear(5, 8)
layer_2 = torch.nn.Linear(8, 4)
layer_3 = torch.nn.Linear(4, 7)

t_output = layer_1(t_input)
t_output = layer_2(t_output)
t_output = layer_3(t_output)

print("Input shape:", t_input.shape)
print("Output shape:", t_output.shape)

Input shape: torch.Size([1, 5])
Output shape: torch.Size([1, 7])


#### Functii de activare

Functiile de activare (activation functions / non-linearities) sunt functii aplicate asupra outputului unui layer linear. Motivul pentru care avem nevoie de astfel de functii este faptul ca un layer transforma inputul aplicand o transformare afina ($A \cdot x + b$). Mai multe transformari afine inlantuite sunt echivalente cu o singura transformare (exista un layer `Linear(5, 7)` care ar face aceeasi transformare de mai sus).

Astfel adaugand mai multe layere intermediare nu crestem cu adevarat complexitatea modelului. În cazul de mai sus, fără funcție de activare între straturi, modelul are aceeasi complexitate ca și când ar avea un singur strat Linear, și este foarte probabil să facă [underfitting](https://www.ibm.com/cloud/learn/underfitting). Un model care nu este suficient de complex pentru task și nu reușește să învețe este un model **underfit**.

Functiile de activare sunt functii nelineare derivabile in toate punctele (eventual cu exceptia unui numar finit de puncte). Functii de activare uzuale sunt [ReLU](https://pytorch.org/docs/1.11/generated/torch.nn.ReLU.html#torch.nn.ReLU) si [tanh](https://pytorch.org/docs/1.11/generated/torch.nn.Tanh.html).

In [39]:
t_input = torch.tensor([[1., 2., 3., 4., 5.]])  # intotdeauna avem shape (batch_size, dim)

layer_1 = torch.nn.Linear(5, 8)
layer_2 = torch.nn.Linear(8, 4)
layer_3 = torch.nn.Linear(4, 7)

t_output = layer_1(t_input)
t_output = torch.nn.functional.relu(t_output)
t_output = layer_2(t_output)
t_output = torch.nn.functional.relu(t_output)
t_output = layer_3(t_output)

print("Input shape:", t_input.shape)
print("Output shape:", t_output.shape)

Input shape: torch.Size([1, 5])
Output shape: torch.Size([1, 7])


Putem folosi layerul `Sequential` pentru a combina mai multe layere intr-unul singur

In [40]:
t_input = torch.tensor([[1., 2., 3., 4., 5.]])  # intotdeauna avem shape (batch_size, dim)

seq = torch.nn.Sequential(
    torch.nn.Linear(5, 8),
    torch.nn.ReLU(),
    torch.nn.Linear(8, 4),
    torch.nn.ReLU(),
    torch.nn.Linear(4, 7),
)

t_output = seq(t_input)

print("Input shape:", t_input.shape)
print("Output shape:", t_output.shape)

Input shape: torch.Size([1, 5])
Output shape: torch.Size([1, 7])


### Dropout

Dropout-ul este o metoda de regularizare a retelelor neuronale. Acesta poate ajuta la antrenarea unor retele mai robuste care pot generaliza mai bine. Principiul de functionare este simplu:
 - la antrenare, fiecare componenta a unui vector trecut prin acest layer este transformata in $0$ cu probabilitate $p$, iar restul componentelor sunt scalate cu $\frac{1}{1-p}$
 - la inferenta (evaluare), acest layer nu afecteaza in niciun fel inputul
 
Intrucat acest layer are comportament diferit la antrenare fata de evaluare, este important sa avem grija ca modelul sa fie setat in prealabil in configuratia corecta (`model.train()` sau `model.eval()`).

In [41]:
t = torch.tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.])
dropout = torch.nn.Dropout(p=0.5)

print("Input:", t)
output = dropout(t)
print("Output:", output)
output = dropout(t)
print("Output:", output)

Input: tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.])
Output: tensor([ 2.,  0.,  0.,  0.,  0., 12.,  0., 16., 18.])
Output: tensor([ 0.,  0.,  6.,  8.,  0., 12.,  0., 16.,  0.])


### Salvarea unui model

In [42]:
torch.save(model.state_dict(), "model.pt")  # salveaza weight-urile modelului
model.load_state_dict(torch.load("model.pt"))  # incarca weight-urile din fisier

<All keys matched successfully>

# TASK

### Deadline: 24 martie ora 23:59.
### Formular pentru trimiterea temei: https://forms.gle/LKCMJmyhS8Z8m7Rc9

1. Porniti de la un dataset artificial generat cu ajutorul metodei [`make_classification`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html) din sklearn (minim 10000 de exemple, cu cel putin 100 de feature-uri relevante, grupate in cel putin 3 clase) sau puteti folosi un toy-dataset de [aici](https://scikit-learn.org/stable/datasets/toy_dataset.html).

2. Definiti un model cu cel putin 3 layere lineare, dintre care ultimul are dimensiunea outputului egala cu numarul de clase. Aplicati dupa fiecare layer linear (cu exceptia ultimului) o functie de activare aleasa de voi.

3. Impartiti datasetul in 80% train, 10% validare si 10% test. Creati cele trei dataloadere corespunzatoare fiecarui split.

4. Definiti functia de loss (cross-entropy) si un optimizer (SGD, Adam, etc.).

5. Antrenati modelul pentru mai multe epoci pe datele de train. La finalul fiecarei epoci evaluati performanta modelului pe datele de validare. Monitorizati la fiecare epoca eroarea medie si acuratetea pentru predictiile facute pe datele de train si separat pe datele de validare, pentru ca in cazul in care observati situatia de *overfit* sa puteti opri antrenarea.

6. Salvati modelul cu cea mai buna eroare de validare, calculata la finalul epocii respective.

7. Plotati pe acelasi grafic evolutia erorii de train si a erorii de validare la finalul fiecarei epoci. Plotati in alt grafic evolutia acuratetii pe datele de train si pe cele de validare.

8. Evaluati modelul (eroare, acuratete, macro-f1, etc.) pe datele de test.

9. Incercati sa adaugati dupa fiecare layer din model (cu exceptia ultimului) dropout cu un $p$ ales de voi. Analizati daca performanta unui model antrenat astfel este mai buna.